In [0]:
# hashlab - google colab powered hash cracking

# cracking tools
hashcat = False
john = False
hydra = False

# shell (only one, or none for colab shell)
ssh = False # uses ngrok, takes a little time to setup
python_shell = False # not interactive
bash_shell = False # don't see input

# wordlists
wordlists_dir = "wordlists"

rockyou = False # generic, lightweight
hashesorg2019 = False # very heavy, but good collection


In [0]:
# wordlists
import os

os.system("wordlists_dir={}".format(wordlists_dir))
!mkdir ./$wordlists_dir
if rockyou:
    !printf "downloading the rockyou wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/90/rockyou.txt.gz
    !printf "wordlist downloaded, extracting...\n"
    !cd $wordlists_dir && gunzip rockyou.txt.gz
    !printf "finished - $(pwd)/$wordlists_dir/$(ls wordlists | grep rockyou)"

if hashesorg2019:
    !printf "downloading the hashesorg2019 wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/1851/hashesorg2019.gz
    !printf "wordlist downloaded, extracting...\n"
    !cd $wordlists_dir && gunzip hashesorg2019.gz
    !printf "finished - $(pwd)/$wordlists_dir/$(ls wordlists | grep hashesorg2019)"

In [0]:
# install hashcat
if hashcat:
    print("installing hashcat...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/hashcat/hashcat.git && cd hashcat && make -j 8 && make install

In [0]:
# install john
if john:
    print("installing john...")
    !apt-get install john -y

In [0]:
# install hydra
if hydra:
    print("installing hydra...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/vanhauser-thc/thc-hydra.git && cd thc-hydra && ./configure && make && make install

In [0]:
# set up shells
from termcolor import colored
import os

if ssh:
    print("configuring ssh...")
    !pip install git+https://github.com/demotomohiro/remocolab.git
    import remocolab
    remocolab.setupSSHD()

elif python_shell:
    os.system("touch /tmp/cmd && touch /tmp/status")

    template_cmd = "echo -n $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
    os.system("bash -c '{}'".format(template_cmd))
    output = {"cmd": "", "status": ""}
    with open('/tmp/cmd', 'r') as f:
        output["cmd"] = f.read()
    with open('/tmp/status', 'r') as f:
        output["status"] = f.read()
    prefixes = output["status"].split("[")
    path = prefixes[2].replace('\n', '')
    prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
    print("")

    while 1:
        print(prefix, end='')
        cmd = input()
        template_cmd = "cd {} && ".format(path) + "" + cmd + " &> /tmp/cmd ; echo $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
        os.system("bash -c '{}'".format(template_cmd))
        output = {"cmd": "", "status": ""}
        with open('/tmp/cmd', 'r') as f:
            output["cmd"] = f.read()
        with open('/tmp/status', 'r') as f:
            output["status"] = f.read()
        prefixes = output["status"].split("[")
        path = prefixes[2].replace('\n', '')
        prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
        print(output["cmd"])

elif bash_shell:
    !/bin/bash


In [0]:
# !hashcat -h
# !john --help